In [1]:
# Dependencies
import pandas as pd
from bs4 import BeautifulSoup as bs
from splinter import Browser
from webdriver_manager.chrome import ChromeDriverManager

In [2]:
# Setting up splinter
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)

[WDM] - ====== WebDriver manager ======
INFO:WDM:====== WebDriver manager ======
[WDM] - Current google-chrome version is 103.0.5060
INFO:WDM:Current google-chrome version is 103.0.5060
[WDM] - Get LATEST chromedriver version for 103.0.5060 google-chrome
INFO:WDM:Get LATEST chromedriver version for 103.0.5060 google-chrome
[WDM] - There is no [win32] chromedriver for browser 103.0.5060 in cache
INFO:WDM:There is no [win32] chromedriver for browser 103.0.5060 in cache
[WDM] - About to download new driver from https://chromedriver.storage.googleapis.com/103.0.5060.53/chromedriver_win32.zip
INFO:WDM:About to download new driver from https://chromedriver.storage.googleapis.com/103.0.5060.53/chromedriver_win32.zip
[WDM] - Driver has been saved in cache [C:\Users\Ray\.wdm\drivers\chromedriver\win32\103.0.5060.53]
INFO:WDM:Driver has been saved in cache [C:\Users\Ray\.wdm\drivers\chromedriver\win32\103.0.5060.53]


In [3]:
# URLs to scrape
url_news = 'https://redplanetscience.com/'
url_imgs = 'https://spaceimages-mars.com'
url_info = 'https://galaxyfacts-mars.com'
url_hemi = 'https://marshemispheres.com/'

In [4]:
# Working with redplanetscience
browser.visit(url_news)

In [5]:
# HTML object
html_news = browser.html

# Using Beautiful Soup
soup_news = bs(html_news,'html.parser')

# Identifying elements with needed information
article = soup_news.find('div', class_='list_text')

In [6]:
# Saving information to variables
news_title = article.find('div', class_='content_title').text
print(news_title)

news_text = article.find('div', class_='article_teaser_body').text
print(news_text)

NASA Invites Public to Share Excitement of Mars 2020 Perseverance Rover Launch
There are lots of ways to participate in the historic event, which is targeted for July 30.


In [7]:
# Working with spaceimages
browser.visit(url_imgs)

In [8]:
# HTML Object
html_imgs = browser.html

# Using Beautiful Soup
soup_imgs = bs(html_imgs, 'html.parser')

# Identifying elements with needed information
image = soup_imgs.find('img', class_='headerimage fade-in')

In [9]:
# Creating url for featured image, saving to variable
featured_image_url = url_imgs + '/' + image['src']
#print(featured_image_url)

In [10]:
# Scraping tablular data with pandas
html_info = pd.read_html(url_info)

In [11]:
# Empty list for iteration
html_str_info = []

# Iterate through list of table data
for tab in html_info:
    
    # Render tables as html strings, append to empty list
    html_str = tab.to_html()
    #print(html_str)
    html_str_info.append(html_str)

#html_str_info

In [12]:
# Working with marshemispheres
browser.visit(url_hemi)

In [13]:
# HTML Object
html_hemi = browser.html

# Using Beautiful Soup
soup_hemi = bs(html_hemi, 'html.parser')

# Identifying elements with needed information
hemispheres = soup_hemi.find_all('div', class_='description')
print(hemispheres)

[<div class="description">
<a class="itemLink product-item" href="cerberus.html">
<h3>Cerberus Hemisphere Enhanced</h3>
</a>
<span class="subtitle" style="float:left">image/tiff 21 MB</span><span class="pubDate" style="float:right"></span><br/>
<p>Mosaic of the Cerberus hemisphere of Mars projected into point perspective, a view similar to that which one would see from a spacecraft. This mosaic is composed of 104 Viking Orbiter images acquired…</p>
</div>, <div class="description">
<a class="itemLink product-item" href="schiaparelli.html">
<h3>Schiaparelli Hemisphere Enhanced</h3>
</a>
<span class="subtitle" style="float:left">image/tiff 35 MB</span><span class="pubDate" style="float:right"></span><br/>
<p>Mosaic of the Schiaparelli hemisphere of Mars projected into point perspective, a view similar to that which one would see from a spacecraft. The images were acquired in 1980 during early northern…</p>
</div>, <div class="description">
<a class="itemLink product-item" href="syrtis.ht

In [14]:
# Empty list for iteration
hemisphere_image_urls = []

# Iterate through list of results to find image data
for h in hemispheres:
    
    # Save name of image
    img_name = h.find('h3')
    hem_name = img_name.text

    # Save image information
    img_src = 'images/' + hem_name.replace(' Hemisphere Enhanced','_enhanced.tif')
    hem_img = url_hemi + img_src.lower().replace(' ','_')
    #print(img_url)

    # Save name and image url as a key:value pair
    hem_dict = {'title': hem_name, 'img_url': hem_img}

    # Append dictionaries to empty list
    hemisphere_image_urls.append(hem_dict)

hemisphere_image_urls

[{'title': 'Cerberus Hemisphere Enhanced',
  'img_url': 'https://marshemispheres.com/images/cerberus_enhanced.tif'},
 {'title': 'Schiaparelli Hemisphere Enhanced',
  'img_url': 'https://marshemispheres.com/images/schiaparelli_enhanced.tif'},
 {'title': 'Syrtis Major Hemisphere Enhanced',
  'img_url': 'https://marshemispheres.com/images/syrtis_major_enhanced.tif'},
 {'title': 'Valles Marineris Hemisphere Enhanced',
  'img_url': 'https://marshemispheres.com/images/valles_marineris_enhanced.tif'}]

In [15]:
# Exiting browser
browser.quit()